<p style="float:right"> <img src="assets/orange.png" alt="Orange logo" width="40" /> <img src="assets/ulb.jpg" alt="ULB logo" width="40" /> <img src="assets/mlg.png" alt="MLG logo" width="160" /> <img src="assets/innoviris.jpg" alt="Innoviris logo" width="200" /></p>

**_Notebook for the project Machu-Picchu written by Théo Verhelst_**<br/>
_Supervisors at Orange: Denis Mercier, Jeevan Shrestha_<br/>
_Academic supervision: Gianluca Bontempi (ULB MLG)_
# Simulating uplift modeling with normal distributions

In [ ]:
import pickle
import numpy as np
from numpy.polynomial import polynomial
import pandas as pd
from scipy import stats as st
from matplotlib import pyplot as plt
import matplotlib
from joblib import Parallel, delayed
from tqdm.autonotebook import trange, tqdm
from sklearn.linear_model import LogisticRegression
from scipy.optimize import curve_fit

from functions.eval_measures import cf_profit_curve
from functions.simulation_functions import simulate_uplift_norm

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"
matplotlib.rc('text.latex', preamble=r'\usepackage{amsmath}')

Here, the data generating process is
\begin{align}
    X&\sim\mathcal N(0, I) \\
    \varepsilon&\sim\mathcal N(0, 1) \\
    T&\sim\mathrm{Bern}(p) \\
    Y_t &= \mathbb I[\lambda_t^TX+\varepsilon\ge\tau_t]  \quad\text{for }t=0,1\\
    Y &= Y_{\mathbb I[T=1]}.
\end{align}
We know that $\lambda_t^TX\sim\mathcal N(0, \lambda_t^T\lambda_t)$.
Then the conditional distribution of $Y_t$ can be computed as
\begin{align}
    P(Y_t=1\mid X=x) &= P(\lambda_t^Tx+\varepsilon>\tau_t)=P(\varepsilon\ge\tau_t-\lambda_t^Tx)\\
    &=1 - F_{\varepsilon}(\tau_t-\lambda_t^Tx)=F_{\varepsilon}(\lambda_t^Tx-\tau_t).
\end{align}

In [ ]:
def compute_info_rate(S_0, S_1):
    H_Y_0 = st.entropy([np.mean(S_0), 1 - np.mean(S_0)])
    H_Y_1 = st.entropy([np.mean(S_1), 1 - np.mean(S_1)])
    H_Y_0_X = np.mean(st.entropy(np.vstack((S_0, 1 - S_0))))
    H_Y_1_X = np.mean(st.entropy(np.vstack((S_1, 1 - S_1))))
    I_0 = 1 - H_Y_0_X / H_Y_0
    I_1 = 1 - H_Y_1_X / H_Y_1
    return I_0, I_1

### Run the simulation once
As a preliminary step, we run once the simulation and verify the distribution of the generated data.

In [ ]:
N_train = 10000
N_test = 100000
N = N_train + N_test
n = 100

# Generate the parameters
mu = np.ones(n)
Sigma = np.eye(n)
lambda_0 = st.norm.rvs(loc=0, scale=15, size=n) / n
lambda_1 = st.norm.rvs(loc=0, scale=15, size=n) / n

# Generate the data
X = st.multivariate_normal.rvs(mean=mu, cov=Sigma, size=N)
noise = st.norm.rvs(size=N)
Q_0 = np.dot(X, lambda_0) + noise
Q_1 = np.dot(X, lambda_1) + noise

eta_0, eta_1 = 1.12, 0.87
Y_0 = Q_0 > eta_0
Y_1 = Q_1 > eta_1
a = np.mean(~Y_0 & ~Y_1)
b  = np.mean( Y_0 & ~Y_1)
c = np.mean(~Y_0 &  Y_1)
d = np.mean( Y_0 &  Y_1)

print(a, b, c, d)

In [ ]:
plt.scatter(np.dot(X, lambda_0) + noise, np.dot(X, lambda_1) + noise, alpha=0.3)
plt.show()

### Multiple runs with fixed parameters
Here, we use a fixed value for the parameters, except that lambda_0 and lambda_1 are multiplied by a parameter ranging from 1e-2 to 1e1.3 on a logarithmic scale.

In [ ]:
N_train = 1000
N_test = 10000
N = N_train + N_test
n = 10
p = 0.1

seed = 444
#seed = 333
rng = np.random.default_rng(seed)

# Generate the parameters
mu = np.zeros(n)
Sigma = np.eye(n)
lambda_0 = st.norm.rvs(loc=1.2, scale=1, size=n, random_state=rng)
lambda_1 = st.norm.rvs(loc=1, scale=1, size=n, random_state=rng)
print("lambda_0 = ", ", ".join("{:.2f}".format(v) for v in lambda_0))
print("lambda_1 = ", ", ".join("{:.2f}".format(v) for v in lambda_1))

# Generate the data
X = st.multivariate_normal.rvs(mean=mu, cov=Sigma, size=N, random_state=rng)
noise = st.logistic.rvs(size=N, random_state=rng)

Q_0 = np.dot(X, lambda_0) + noise
Q_1 = np.dot(X, lambda_1) + noise
eta_0, eta_1 = 1.12, 0.87
Y_0 = Q_0 > eta_0
Y_1 = Q_1 > eta_1
a = np.mean(~Y_0 & ~Y_1)
b  = np.mean( Y_0 & ~Y_1)
c = np.mean(~Y_0 &  Y_1)
d = np.mean( Y_0 &  Y_1)

print("eta_0 = {:.2f}, eta_1 = {:.2f}".format(eta_0, eta_1))
print("mu = {:.2f}, {:.2f}, {:.2f}, {:.2f}".format(a, b, c, d))

results = []

for scale in tqdm(np.logspace(-2, 1.3, num=100)):
    l_0 = lambda_0 * scale
    l_1 = lambda_1 * scale
    
    # Sometimes the probability of Y_1=y is too low and we have
    # zero sample in this class. Let's try again in this case.
    while True:
        try:
            auuc_u, auuc_p = simulate_uplift_norm(N_train, X, noise, l_0, l_1, eta_0, eta_1, p, 1000, rng)
            break
        except ValueError:
            continue
    
    # Use more data to compute the information rate
    N_p = N * 10
    X_p = st.multivariate_normal.rvs(mean=mu, cov=Sigma, size=N_p, random_state=rng)
    S_0_p = st.logistic.cdf(np.dot(X_p, l_0))
    S_1_p = st.logistic.cdf(np.dot(X_p, l_1))
    
    I_0, I_1 = compute_info_rate(S_0_p, S_1_p)
    results.append({
        "scale": scale,
        "auuc_u": auuc_u,
        "auuc_p": auuc_p,
        "I_0": I_0,
        "I_1": I_1
    })

In [ ]:
plt.rcParams["figure.figsize"] = (5.9, 3.15)

df = pd.DataFrame.from_records(results)
plt.plot(df.I_0, df.auuc_u * 100, label="Uplift approach", color="black", linestyle="solid")
plt.plot(df.I_0, df.auuc_p * 100, label="Predictive approach", color="black", linestyle="dashed")
plt.ylabel("AUPC (%)")
plt.xscale("log")
plt.xlabel(r"$I({\bf x}; {\bf y}_0)\;/\;H({\bf y}_0)$")
plt.legend()

#plt.savefig("pdf/simulation_norm.pdf", bbox_inches="tight", dpi=240)
plt.show()

### Multiple runs with random parameters
Here, we randomize each parameter each run.

In [ ]:
N_train = 1000
N_test = 10000
scale_space_size = 100
n = 10
p = 0.04
n_repeats = 100

seed = 444
rng = np.random.default_rng(seed)

def run_normal_dis(i, N_train, N_test, scale_space_size, n, p, rng):
    rng = None
    # Generate the parameters
    N = N_train + N_test
    mu = np.zeros(n)
    Sigma = np.eye(n)
    lambda_0 = st.norm.rvs(loc=1.2, scale=1, size=n, random_state=rng)
    lambda_1 = st.norm.rvs(loc=1, scale=1, size=n, random_state=rng)

    # Generate the data
    X = st.multivariate_normal.rvs(mean=mu, cov=Sigma, size=N, random_state=rng)
    noise = st.logistic.rvs(size=N, random_state=rng)

    # Find the thresholds that closely match the desired counterfactual distribution
    alpha, beta, gamma, delta = 0.4, 0.2, 0.2, 0.2
    #alpha, beta, gamma, delta = 0.7, 0.1, 0.1, 0.1
    Q_0 = np.dot(X, lambda_0) + noise
    Q_1 = np.dot(X, lambda_1) + noise
    eta_0, eta_1 = 1.12, 0.87
    Y_0 = Q_0 > eta_0
    Y_1 = Q_1 > eta_1
    a = np.mean(~Y_0 & ~Y_1)
    b  = np.mean( Y_0 & ~Y_1)
    c = np.mean(~Y_0 &  Y_1)
    d = np.mean( Y_0 &  Y_1)

    results = []
    for scale in np.logspace(-2, 1.3, num=scale_space_size):
        l_0 = lambda_0 * scale
        l_1 = lambda_1 * scale

        # Sometimes the probability of Y_1=y is too low and we have
        # zero sample in this class. Let's try again in this case.
        T = 0
        while True:
            try:
                auuc_u, auuc_p = simulate_uplift_norm(N_train, X, noise, l_0, l_1, eta_0, eta_1, p, 1000, rng)
                break
            except ValueError:
                T += 1
                if T >= 1000:
                    break
                continue
        if T >= 1000:
            continue

        # Use more data to compute the information rate
        N_p = N * 10
        X_p = st.multivariate_normal.rvs(mean=mu, cov=Sigma, size=N_p, random_state=rng)
        S_0_p = st.logistic.cdf(np.dot(X_p, l_0))
        S_1_p = st.logistic.cdf(np.dot(X_p, l_1))

        I_0, I_1 = compute_info_rate(S_0_p, S_1_p)
        results.append({
            "i": i,
            "scale": scale,
            "auuc_u": auuc_u,
            "auuc_p": auuc_p,
            "I_0": I_0,
            "I_1": I_1
        })
    return results

results = Parallel(n_jobs=4)(
    delayed(run_normal_dis)(
        i, N_train, N_test, scale_space_size, n, p, rng
    ) for i in trange(n_repeats)
)
results = sum(results, [])

stats = pd.DataFrame.from_records(results)

These two functions are used to create the plot below. The AUPC is plotted as a function of the mutual information by fitting a 4th-degree polynomial. Then, we use a moving Gaussian kernel to estimate locally the standard deviation of the AUPC.

In [ ]:
def weigthed_sample_std(x, x_bar, weights):
    weights /= weights.sum()
    squared_error = np.dot((x - x_bar)**2, weights)
    N = x.shape[0]
    return np.sqrt(squared_error * N / (N - 1))

def confidence_curves(x, y, fitted_x, fitted_y, bw, alpha):
    cb_min = []
    cb_max = []
    N = len(fitted_x)
    for i in range(N):
        # Use a gaussian window on the data points
        weights = st.norm.pdf(x, loc=fitted_x[i], scale=bw)
        # Fix the location to the predicted value
        mean = fitted_y[i]
        sample_var = weigthed_sample_std(y, mean, weights)
        #_, sample_var = st.norm.fit(y_w, loc=mean)
        z = (1 - alpha) / 2
        cb_min.append(st.norm.ppf(z, loc=mean, scale=sample_var))
        cb_max.append(st.norm.ppf(1 - z, loc=mean, scale=sample_var))
    return cb_min, cb_max

In [ ]:
x = stats.I_0
y_u = stats.auuc_u * 100
y_p = stats.auuc_p * 100
P_u = polynomial.Polynomial.fit(x=x, y=y_u, deg=4)
P_p = polynomial.Polynomial.fit(x=x, y=y_p, deg=4)
space_u = P_u.linspace()
space_p = P_p.linspace()
cb_min_u, cb_max_u = confidence_curves(x, y_u, space_u[0], space_u[1], 0.01, 0.95)
cb_min_p, cb_max_p = confidence_curves(x, y_p, space_p[0], space_p[1], 0.01, 0.95)

In [ ]:
plt.rcParams["figure.figsize"] = (5.9, 3.15)
cmap = {
    "u": (31, 119, 180),
    "p": (255,127,14),
    "e": (148,103,189)
}
def rgb_to_hex(rgb):
    return tuple(c/255 for c in rgb)

#plt.scatter(x, y_u, label="Uplift approach", color=rgb_to_hex(cmap["u"]), alpha=0.02)
plt.plot(space_u[0], space_u[1], color=rgb_to_hex(cmap["u"]), label="Uplift")
plt.fill_between(space_u[0], cb_min_u, cb_max_u, color=rgb_to_hex(cmap["u"]), alpha=0.2)
#plt.scatter(x, y_p, label="Predictive approach", color=rgb_to_hex(cmap["p"]), alpha=0.02)
plt.plot(space_p[0], space_p[1], color=rgb_to_hex(cmap["p"]), label="Predictive")
plt.fill_between(space_p[0], cb_min_p, cb_max_p, color=rgb_to_hex(cmap["p"]), alpha=0.2)
plt.ylabel("AUPC (%)")
plt.xscale("log")
plt.xlabel(r"$I({\bf x}; {\bf y}_0)\;/\;H({\bf y}_0)$")
plt.legend(title="Approach", loc="upper left")

#plt.savefig("pdf/simulation_norm_repeated.pdf", bbox_inches="tight", dpi=240)
plt.show()